In [1]:
!python3 -m pip install --upgrade pip
!git clone -b mixtral https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Cloning into 'mergekit'...
remote: Enumerating objects: 2002, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 2002 (delta 72), reused 66 (delta 42), pack-reused 1886
Receiving objects: 100% (2002/2002), 613.01 KiB | 5.68 MiB/s, done.
Resolving deltas: 100% (1368/1368), done.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.9.0 requires keras-core, which is not installed.
keras-nlp 0.12.1 requires keras-core, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.2.1 which is incompatible.
conda 24.5.0 

In [2]:
from huggingface_hub import notebook_login as hfl
hfl()

In [9]:
MODEL_NAME = "Code-Mistral-Llama-7B-TIES"
OUTPUT_DIR = "merge"
CONFIG_NAME = "config"

In [10]:
config = """
models:
  - model: mistralai/Mistral-7B-v0.1  # no parameters necessary for base model
  - model: codellama/CodeLlama-7b-Instruct-hf
    parameters:
      density: 0.5
      weight: 0.5
merge_method: ties
base_model: mistralai/Mistral-7B-v0.1
parameters:
  normalize: true
  int8_mask: true
dtype: float16
"""

In [11]:
with open(f"{CONFIG_NAME}.yaml", "w") as f:
    f.write(config)

In [12]:
mergeCommand = f"mergekit-yaml {CONFIG_NAME}.yaml {OUTPUT_DIR} --allow-crimes --copy-tokenizer --out-shard-size 1B --low-cpu-memory --write-model-card --lazy-unpickle"  

In [13]:
!{mergeCommand}

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 10 files:   0%|                                 | 0/10 [00:00<?, ?it/s]

pytorch_model.bin.index.json: 100%|████████| 23.9k/23.9k [00:00<00:00, 82.2MB/s]


generation_config.json: 100%|███████████████████| 116/116 [00:00<00:00, 729kB/s]


model.safetensors.index.json:   0%|                 | 0.00/25.1k [00:00<?, ?B/s]
model.safetensors.index.json: 100%|████████| 25.1k/25.1k [00:00<00:00, 6.81MB/s]


special_tokens_map.json: 100%|████████████████| 72.0/72.0 [00:00<00:00, 498kB/s]


tokenizer.json:   0%|                               | 0.00/1.80M [00:00<?, ?B/s]


model-00002-of-00002.safetensors:   0%|             | 0.00/4.54G [00:00<?, ?B/s]



tokenizer.json: 100%|██████████████████████| 1.80M/1.8

In [15]:
from kaggle_secrets import UserSecretsClient

userSecrets = UserSecretsClient()
HF_WRITE_TOKEN = userSecrets.get_secret("HF_WRITE_TOKEN")

In [16]:
from huggingface_hub import HfApi

USER_NAME = "0xVolt"

# Defined in the secrets tab in Kaggle Secrets
api = HfApi(token=HF_WRITE_TOKEN)

api.create_repo(
    repo_id=f"{USER_NAME}/{MODEL_NAME}",
    repo_type="model"
)

# Push the whole merged model's folder to the hub
api.upload_folder(
    repo_id=f"{USER_NAME}/{MODEL_NAME}",
    folder_path=OUTPUT_DIR,
)

model-00004-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Upload 9 LFS files:   0%|          | 0/9 [00:00<?, ?it/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/906M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/0xVolt/Code-Mistral-Llama-7B-TIES/commit/7e8b16d83267f068a8ccc6aec163be6d3f56c6bc', commit_message='Upload folder using huggingface_hub', commit_description='', oid='7e8b16d83267f068a8ccc6aec163be6d3f56c6bc', pr_url=None, pr_revision=None, pr_num=None)